In [1]:
import numpy
import xarray
import glob
import pandas
import cartopy
import itertools
import matplotlib
import cmocean
import netCDF4
import datetime
import cftime

import matplotlib.pyplot as mp

mp.rcParams.update({'mathtext.default': 'regular'})
%matplotlib inline

In [4]:
PRECC_orig = '/glade/p_old/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly/PRECC/b.e11.B1850C5CN.f09_g16.005.cam.h0.PRECC.040001-049912.nc'
PRECC_orig_ds = xarray.open_dataset(PRECC_orig)
lat_full = PRECC_orig_ds['lat'].values
lon_full = PRECC_orig_ds['lon'].values

In [5]:
PRECC_root = '/glade/p_old/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly/PRECC/'
PRECL_root = '/glade/p_old/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly/PRECL/'

QSOIL_root = '/glade/p_old/cesmLE/CESM-CAM5-BGC-LE/lnd/proc/tseries/monthly/QSOIL/'
QVEGE_root = '/glade/p_old/cesmLE/CESM-CAM5-BGC-LE/lnd/proc/tseries/monthly/QVEGE/'
QVEGT_root = '/glade/p_old/cesmLE/CESM-CAM5-BGC-LE/lnd/proc/tseries/monthly/QVEGT/'

LHFLX_root = '/glade/p_old/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly/LHFLX/'

In [8]:
pic_file_list_PRECC = numpy.array(sorted(glob.glob(PRECC_root+'/b.e11.B1850C5CN.f09_g16.*.nc')))
pic_file_list_PRECL = numpy.array(sorted(glob.glob(PRECL_root+'/b.e11.B1850C5CN.f09_g16.*.nc')))

pic_file_list_QSOIL = numpy.array(sorted(glob.glob(QSOIL_root+'/b.e11.B1850C5CN.f09_g16.*.nc')))
pic_file_list_QVEGE = numpy.array(sorted(glob.glob(QVEGE_root+'/b.e11.B1850C5CN.f09_g16.*.nc')))
pic_file_list_QVEGT = numpy.array(sorted(glob.glob(QVEGT_root+'/b.e11.B1850C5CN.f09_g16.*.nc')))
pic_file_list_LHFLX = numpy.array(sorted(glob.glob(LHFLX_root+'/b.e11.B1850C5CN.f09_g16.*.nc')))

In [9]:
rcp_file_list_PRECC = numpy.array(sorted(glob.glob(PRECC_root+'/b.e11.BRCP85C5CNBDRD.f09_g16.*.nc')))
rcp_file_list_PRECL = numpy.array(sorted(glob.glob(PRECL_root+'/b.e11.BRCP85C5CNBDRD.f09_g16.*.nc')))

rcp_file_list_QSOIL = numpy.array(sorted(glob.glob(QSOIL_root+'/b.e11.BRCP85C5CNBDRD.f09_g16.*.nc')))
rcp_file_list_QVEGE = numpy.array(sorted(glob.glob(QVEGE_root+'/b.e11.BRCP85C5CNBDRD.f09_g16.*.nc')))
rcp_file_list_QVEGT = numpy.array(sorted(glob.glob(QVEGT_root+'/b.e11.BRCP85C5CNBDRD.f09_g16.*.nc')))
rcp_file_list_LHFLX = numpy.array(sorted(glob.glob(LHFLX_root+'/b.e11.BRCP85C5CNBDRD.f09_g16.*.nc')))

# open pic files

In [42]:
PRECC_pic_mfds = xarray.open_mfdataset(pic_file_list_PRECC, decode_times=False)
PRECL_pic_mfds = xarray.open_mfdataset(pic_file_list_PRECL, decode_times=False)

QSOIL_pic_mfds = xarray.open_mfdataset(pic_file_list_QSOIL, decode_times=False)
QVEGE_pic_mfds = xarray.open_mfdataset(pic_file_list_QVEGE, decode_times=False)
QVEGT_pic_mfds = xarray.open_mfdataset(pic_file_list_QVEGT, decode_times=False)
#LHFLX_pic_mfds = xarray.open_mfdataset(pic_file_list_LHFLX, decode_times=False) # need to divide by 2.256e6 kJ/kg

In [ ]:
QTOT_pic_mfds = QSOIL_pic_mfds['QSOIL']+QVEGE_pic_mfds['QVEGE']+QVEGT_pic_mfds['QVEGT']
QTOT_pic = QTOT_pic_mfds.values*86400.

In [ ]:
PRECT_pic = (PRECC_pic_mfds['PRECC']+PRECL_pic_mfds['PRECL']).values*86400.*1000.

In [ ]:
pic_time_datetime = numpy.array([cftime.DatetimeNoLeap(year,month,1) \
                                 for year,month in itertools.product(range(400,2201), range(1, 13))])

# open hist/rcp files

In [ ]:
# define member strings for hist/rcp
multifile_members = ['{:03d}'.format(i) for i in range(1,34)]
all_members = ['{:03d}'.format(i) for i in range(1,36)] + ['{:03d}'.format(i) for i in range(101,106)]

In [ ]:
rcp_time_datetime = numpy.array([cftime.DatetimeNoLeap(year,month,1) \
                                 for year,month in itertools.product(range(2006,2101), range(1,13))])

In [ ]:
rcp_start = cftime.DatetimeNoLeap(2070,1,1)
rcp_end = cftime.DatetimeNoLeap(2099,12,31)
rcp_time_indices = list(numpy.where((rcp_time_datetime>=rcp_start)&(rcp_time_datetime<=rcp_end))[0])

In [ ]:
rcp_nyears = 30
rcp_year_array = numpy.arange(rcp_start.year, rcp_end.year+1, dtype=numpy.int)
rcp_time_datetime_subset = rcp_time_datetime[rcp_time_indices]

In [ ]:
PRECT_all_rcp_data = numpy.zeros((all_members.__len__(), rcp_nyears*12, lat_full.size, lon_full.size))

In [ ]:
# do all this for the first lon box to get the time and lat/lon information
for member_idx in range(40):
    
    which_member = all_members[member_idx]
    #print(which_member)

    PRECC_file_indices = [file.split('/')[-1].split('.')[4]==which_member for file in rcp_file_list_PRECC]
    PRECL_file_indices = [file.split('/')[-1].split('.')[4]==which_member for file in rcp_file_list_PRECL]

    PRECC_member_file_list = rcp_file_list_PRECC[PRECC_file_indices]
    PRECL_member_file_list = rcp_file_list_PRECL[PRECL_file_indices]

    if PRECC_member_file_list.__len__()!=PRECL_member_file_list.__len__():
        print('weird not same length')
    elif PRECC_member_file_list.__len__()>1:
        PRECC_rcp_ds = xarray.open_mfdataset(PRECC_member_file_list)
        PRECL_rcp_ds = xarray.open_mfdataset(PRECL_member_file_list)
    else:
        PRECC_rcp_ds = xarray.open_dataset(PRECC_member_file_list[0])
        PRECL_rcp_ds = xarray.open_dataset(PRECL_member_file_list[0])

    # prect_rcp = prect_rcp_ds['PRECT']*86400.*1000.

    PRECT_rcp_ds = (PRECC_rcp_ds['PRECC']+PRECL_rcp_ds['PRECL']).isel(time=rcp_time_indices)
    PRECT_all_rcp_data[member_idx,:,:,:] = PRECT_rcp_ds.values*86400.*1000.

# doing seasonal means for pic

In [ ]:
window_length=6 # ONDJFM, AMJJAS

In [ ]:
# calculate rolling mean
# pick off the window that matters
# take seasonal means of data set
pic_nyears = pic_time_datetime[-1].year - pic_time_datetime[0].year
pic_year_start = pic_time_datetime[0].year
pic_year_end = pic_time_datetime[-1].year
pic_year_array = numpy.arange(pic_year_start, pic_year_start+pic_nyears, dtype=numpy.int)

PRECT_pic_rolling_means = numpy.zeros((PRECT_pic.shape))
for j in range(lat_full.size):
    if j%50==0:
        print(j)
    for k in range(lon_full.size):
        PRECT_pic_rolling_means[:,j,k] = pandas.Series(PRECT_pic[:,j,k]).rolling(window=window_length, min_periods=1).mean()

#

# take moving window means of hist

In [ ]:
PRECT_rcp_rolling_means = numpy.zeros((PRECT_all_rcp_data.shape))
for member_idx in range(40):
    print(member_idx)
    for j in range(lat_full.size):
        for k in range(lon_full.size):
            PRECT_rcp_rolling_means[member_idx,:,j,k] = pandas.Series(PRECT_all_rcp_data[member_idx,:,j,k]).rolling(window=window_length, min_periods=1).mean()

In [ ]:
SEAS_list = [10,11,12,1,2,3]; SEAS_name = 'ONDJFM'; window_length=6
#SEAS_list = [4,5,6,7,8,9]; SEAS_name = 'AMJJAS'; split=False
#SEAS_list = [12,1,2]; SEAS_name = 'DJF'; window_length=3

# pull out pic seasons

In [ ]:
pic_seas_endmonth_indices = [(d.month==SEAS_list[-1]) for d in pic_time_datetime]
PRECT_pic_seas_means = PRECT_pic_rolling_means[pic_seas_endmonth_indices,:,:][1:-1,:,:]

# calculating return periods of seasonal means

In [ ]:
return_period = 100 # in years

In [ ]:
events_per_year = 1
return_val_perc = 100*(1-1/(return_period*events_per_year))
print(return_val_perc)

pic_hi_percentile_values = numpy.zeros((lat_full.size,lon_full.size))
for j in range(lat_full.size):
    for k in range(lon_full.size):
        tmp_distro = PRECT_pic_seas_means[:,j,k]
        pic_hi_percentile_values[j,k] = numpy.percentile(tmp_distro, return_val_perc)

In [ ]:
events_per_year = 1
return_val_perc = 100*(1/(return_period*events_per_year))
print(return_val_perc)

pic_lo_percentile_values = numpy.zeros((lat_full.size,lon_full.size))
for j in range(lat_full.size):
    for k in range(lon_full.size):
        tmp_distro = PRECT_pic_seas_means[:,j,k]
        pic_lo_percentile_values[j,k] = numpy.percentile(tmp_distro, return_val_perc)

# plot return period values of pic

In [ ]:
fontsize=12

map_proj = cartopy.crs.PlateCarree(central_longitude=180)
data_proj = cartopy.crs.PlateCarree(central_longitude=0)

fig = mp.figure(figsize=(8.5,5))
ax = fig.add_subplot(111, projection=map_proj)
ax.coastlines()

ax.text(s='PIC 100-year '+SEAS_name+' return value', x=0.5, y=1.02, ha='center', va='bottom', fontsize=fontsize, \
       transform=ax.transAxes)

clevels = numpy.arange(0,21,2.5)
plot = ax.contourf(lon_full, lat_full, pic_hi_percentile_values, \
                   levels=clevels, \
                   transform=data_proj, \
                   cmap='magma_r', extend='max')

fig.tight_layout()

axpos = ax.get_position()
cbar_ax = fig.add_axes([axpos.x0, axpos.y0-.1, axpos.width, 0.05])
cbar_ax.tick_params(labelsize=fontsize)

cbar = fig.colorbar(plot, orientation='horizontal',cax=cbar_ax)
cbar.set_label('mm day$^{\,-1}$', fontsize=fontsize)

In [ ]:
fontsize=12

map_proj = cartopy.crs.PlateCarree(central_longitude=180)
data_proj = cartopy.crs.PlateCarree(central_longitude=0)

fig = mp.figure(figsize=(8.5,5))
ax = fig.add_subplot(111, projection=map_proj)
ax.coastlines()

ax.text(s='PIC 100-year '+SEAS_name+' return value', x=0.5, y=1.02, ha='center', va='bottom', fontsize=fontsize, \
       transform=ax.transAxes)

clevels = numpy.arange(0,21,2.5)
plot = ax.contourf(lon_full, lat_full, pic_lo_percentile_values, \
                   levels=clevels, \
                   transform=data_proj, \
                   cmap='magma_r', extend='max')

fig.tight_layout()

axpos = ax.get_position()
cbar_ax = fig.add_axes([axpos.x0, axpos.y0-.1, axpos.width, 0.05])
cbar_ax.tick_params(labelsize=fontsize)

cbar = fig.colorbar(plot, orientation='horizontal',cax=cbar_ax)
cbar.set_label('mm day$^{\,-1}$', fontsize=fontsize)

# pull out rcp seasons

In [ ]:
rcp_seas_endmonth_indices = [(d.month==SEAS_list[-1]) for d in rcp_time_datetime_subset]
PRECT_rcp_seas_means = numpy.zeros((all_members.__len__(), sum(rcp_seas_endmonth_indices), lat_full.size, lon_full.size))

for member_idx in range(40):
    PRECT_rcp_seas_means[member_idx,:,:,:] = PRECT_rcp_rolling_means[member_idx,rcp_seas_endmonth_indices,:,:]
PRECT_rcp_seas_means = PRECT_rcp_seas_means[:,1:-1,:,:]

# count exceedances using pic calculation

In [ ]:
PRECT_rcp_exceedance_counts = numpy.zeros((lat_full.size, lon_full.size))

for j in range(lat_full.size):
    for k in range(lon_full.size):
        PRECT_rcp_exceedance_counts[j,k] = numpy.sum(PRECT_rcp_seas_means[:,:,j,k]>pic_hi_percentile_values[j,k])

In [ ]:
PRECT_rcp_deceedance_counts = numpy.zeros((lat_full.size, lon_full.size))

for j in range(lat_full.size):
    for k in range(lon_full.size):
        PRECT_rcp_deceedance_counts[j,k] = numpy.sum(PRECT_rcp_seas_means[:,:,j,k]<pic_lo_percentile_values[j,k])

In [ ]:
rcp_nyears_seas_means = PRECT_rcp_seas_means.shape[1]
pic_nyears_seas_means = PRECT_pic_seas_means.shape[0]

In [ ]:
rcp_nyears_seas_means

In [ ]:
pic_nyears_seas_means

In [ ]:
PRECT_rcp_normalized_exceedances = PRECT_rcp_exceedance_counts/(rcp_nyears_seas_means*40)/((pic_nyears_seas_means/return_period)/pic_nyears_seas_means)
PRECT_rcp_normalized_deceedances = PRECT_rcp_deceedance_counts/(rcp_nyears_seas_means*40)/((pic_nyears_seas_means/return_period)/pic_nyears_seas_means)


In [ ]:
# make color map
minval=0.0 
maxval=0.95
n=256
full_cmap = cmocean.cm.balance_r
full_cmap_r = cmocean.cm.balance
cmap_partial = matplotlib.colors.LinearSegmentedColormap.from_list('trunc({n},{a:.2f},{b:.2f})'.format(n=full_cmap.name, a=minval, b=maxval), full_cmap(numpy.linspace(minval, maxval, n)))
cmap_partial_r = matplotlib.colors.LinearSegmentedColormap.from_list('trunc({n},{a:.2f},{b:.2f})'.format(n=full_cmap_r.name, a=minval, b=maxval), full_cmap_r(numpy.linspace(minval, maxval, n)))

In [ ]:
#clevels = numpy.hstack((numpy.arange(0,1.1,0.1), numpy.arange(5,50.1,5)))
clevels = numpy.hstack((numpy.arange(0,1.1,0.1), numpy.array([2,3,4,5,10,15,20,30,50,100])))
bounds = clevels
norm = matplotlib.colors.BoundaryNorm(boundaries=bounds, ncolors=256)

In [ ]:
clevels

In [ ]:
fontsize=12

map_proj = cartopy.crs.PlateCarree(central_longitude=0)
data_proj = cartopy.crs.PlateCarree(central_longitude=0)

fig = mp.figure(figsize=(8.5,5))
ax = fig.add_subplot(111, projection=map_proj)
ax.coastlines(color='0.9')

ax.text(s='2070-2100 100-year '+SEAS_name+'\nreturn value frequency change', \
        x=0.5, y=1.02, ha='center', va='bottom', fontsize=fontsize, \
        transform=ax.transAxes)

plot = ax.contourf(lon_full, lat_full, PRECT_rcp_normalized_exceedances, \
                   levels=clevels, \
                   transform=data_proj, extend='max', \
                   cmap=cmap_partial, norm=norm)

#plot = ax.pcolormesh(lon_full, lat_full, PRECT_rcp_normalized_exceedances, vmin=0, vmax=2, cmap='RdBu', transform=data_proj)

plot_one = ax.contour(lon_full, lat_full, PRECT_rcp_normalized_exceedances, \
                      levels=[1.0], linewidths=[3], colors=['0.1'], transform=data_proj)

fig.tight_layout()

axpos = ax.get_position()
cbar_ax = fig.add_axes([axpos.x0, axpos.y0-.1, axpos.width, 0.05])
cbar_ax.tick_params(labelsize=fontsize)

cbar = fig.colorbar(plot, orientation='horizontal',cax=cbar_ax)
cbar.set_ticks([0,1,2,3,4,5,10,15,20,30,50,100])
cbar.set_label('change in likelihood', fontsize=fontsize)

In [ ]:
fontsize=12

map_proj = cartopy.crs.PlateCarree(central_longitude=0)
data_proj = cartopy.crs.PlateCarree(central_longitude=0)

fig = mp.figure(figsize=(8.5,5))
ax = fig.add_subplot(111, projection=map_proj)
ax.coastlines(color='0.9')

ax.text(s='2070-2100 100-year '+SEAS_name+'\nreturn value frequency change', \
        x=0.5, y=1.02, ha='center', va='bottom', fontsize=fontsize, \
        transform=ax.transAxes)

plot = ax.contourf(lon_full, lat_full, PRECT_rcp_normalized_deceedances, \
                   levels=clevels, \
                   transform=data_proj, extend='max', \
                   cmap=cmap_partial_r, norm=norm)

plot_one = ax.contour(lon_full, lat_full, PRECT_rcp_normalized_deceedances, \
                      levels=[1.0], linewidths=[3], colors=['0.1'], transform=data_proj)

fig.tight_layout()

axpos = ax.get_position()
cbar_ax = fig.add_axes([axpos.x0, axpos.y0-.1, axpos.width, 0.05])
cbar_ax.tick_params(labelsize=fontsize)

cbar = fig.colorbar(plot, orientation='horizontal',cax=cbar_ax)
cbar.set_ticks([0,1,2,3,4,5,10,15,20,30,50,100])
cbar.set_label('change in likelihood', fontsize=fontsize)

In [ ]:
threshold = 1.5

In [ ]:
sign_switch_regions = numpy.array((PRECT_rcp_normalized_deceedances>threshold)&(PRECT_rcp_normalized_exceedances>threshold), dtype=numpy.float)
sign_switch_regions[sign_switch_regions==0]=numpy.nan

In [ ]:
fontsize=12

map_proj = cartopy.crs.PlateCarree(central_longitude=0)
data_proj = cartopy.crs.PlateCarree(central_longitude=0)

fig = mp.figure(figsize=(8.5,5))
ax = fig.add_subplot(111, projection=map_proj)
ax.coastlines()

ax.text(s='2070-2100 100-year '+SEAS_name+'\nreturn value frequency change', \
        x=0.5, y=1.02, ha='center', va='bottom', fontsize=fontsize, \
        transform=ax.transAxes)

plot = ax.pcolormesh(lon_full, lat_full, sign_switch_regions, \
                    #levels=clevels, \
                    transform=data_proj, \
                    cmap='Oranges', vmin=0, vmax=1)

fig.tight_layout()

# calculate actual whiplash metric

In [ ]:
# get 20th and 80th percentile of seasonal precip in pic
# then loop through pic and calculate frequency at qhich it switches from one to another
# then do same for rcp

PRECT_pic_80th = numpy.percentile(PRECT_pic_seas_means, q=80, axis=0)
PRECT_pic_20th = numpy.percentile(PRECT_pic_seas_means, q=20, axis=0)

In [ ]:
pic_whiplash_count = numpy.zeros((PRECT_pic_seas_means.shape[1:]))
#for y in range(pic_nyears_seas_means-1):
y=0
while y<pic_nyears_seas_means-1:
    if y%200==0:
        print(y)
    for j in range(lat_full.size):
        for k in range(lon_full.size):
            if (PRECT_pic_seas_means[y,j,k]<PRECT_pic_20th[j,k])&(PRECT_pic_seas_means[y+1,j,k]>PRECT_pic_80th[j,k]):
                pic_whiplash_count[j,k]+=1
    y+=1

In [ ]:
rcp_whiplash_count = numpy.zeros((PRECT_pic_seas_means.shape[1:]))
for member_idx in range(40):
    print(member_idx)
    y=0
    while y<rcp_nyears_seas_means-1:
        for j in range(lat_full.size):
            for k in range(lon_full.size):
                if (PRECT_rcp_seas_means[member_idx,y,j,k]<PRECT_pic_20th[j,k])&(PRECT_rcp_seas_means[member_idx,y+1,j,k]>PRECT_pic_80th[j,k]):
                    rcp_whiplash_count[j,k]+=1
        y+=1

In [ ]:
pic_whiplash_count_normalized = (pic_whiplash_count/pic_nyears_seas_means)*100.
rcp_whiplash_count_normalized = (rcp_whiplash_count/(rcp_nyears_seas_means*40))*100.

In [ ]:
fontsize=12

map_proj = cartopy.crs.PlateCarree(central_longitude=180)
data_proj = cartopy.crs.PlateCarree(central_longitude=0)

fig = mp.figure(figsize=(8.5,5))
ax = fig.add_subplot(111, projection=map_proj)
ax.coastlines(color='0.9')

ax.text(s='Whiplash events per 100yr during '+SEAS_name, x=0.5, y=1.02, ha='center', va='bottom', fontsize=fontsize, \
       transform=ax.transAxes)

clevels = numpy.arange(0,7.1,1)
plot = ax.contourf(lon_full, lat_full, pic_whiplash_count_normalized, \
                   levels=clevels, \
                   transform=data_proj, \
                   cmap='magma', extend='max')

fig.tight_layout()

axpos = ax.get_position()
cbar_ax = fig.add_axes([axpos.x0, axpos.y0-.1, axpos.width, 0.05])
cbar_ax.tick_params(labelsize=fontsize)

cbar = fig.colorbar(plot, orientation='horizontal',cax=cbar_ax)
cbar.set_label('count', fontsize=fontsize)

In [ ]:
fontsize=12

map_proj = cartopy.crs.PlateCarree(central_longitude=180)
data_proj = cartopy.crs.PlateCarree(central_longitude=0)

fig = mp.figure(figsize=(8.5,5))
ax = fig.add_subplot(111, projection=map_proj)
ax.coastlines(color='0.1')

ax.text(s='Ratio of whiplash likelihood '+SEAS_name, x=0.5, y=1.02, ha='center', va='bottom', fontsize=fontsize, \
       transform=ax.transAxes)

clevels = numpy.arange(0,2.1,0.2)
plot = ax.contourf(lon_full, lat_full, rcp_whiplash_count_normalized/pic_whiplash_count_normalized, \
                   levels=clevels, \
                   transform=data_proj, \
                   cmap=cmocean.cm.balance, extend='max')

fig.tight_layout()

axpos = ax.get_position()
cbar_ax = fig.add_axes([axpos.x0, axpos.y0-.1, axpos.width, 0.05])
cbar_ax.tick_params(labelsize=fontsize)

cbar = fig.colorbar(plot, orientation='horizontal',cax=cbar_ax)
cbar.set_ticks([0,0.5,1,1.5,2])
cbar.set_label('ratio', fontsize=fontsize)